In [41]:
import sys; sys.path.append('../')
from DataPreparing.data_preparing import read_data
from Evaluation.evaluate import *
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader


In [42]:
X_train, X_val, y_train, y_val =read_data("../DataFiles/embeddings_121_191_401_457.csv")

In [43]:
# concatenate x_train and x_val
X_train = np.concatenate((X_train, X_val), axis=0)
y_train = np.concatenate((y_train, y_val), axis=0)


In [44]:
# convert float to float64 
X_train = X_train.astype('float64')
X_val = X_val.astype('float64')


In [45]:
print(set   (y_train))
print(set   (y_val))

{'CWE457', 'CWE121', 'CWE191', 'CWE401'}
{'CWE457', 'CWE121', 'CWE191', 'CWE401'}


In [46]:
# encode class values as integers safe=0, CWE23=1
for i in range(len(y_train)):
    # if y_train[i]=='safe':
    #     y_train[i]=0
    if y_train[i]=='CWE121':
        y_train[i]=0
    elif y_train[i]=='CWE191':
        y_train[i]=1
    elif y_train[i]=='CWE401':
        y_train[i]=2
    elif y_train[i]=='CWE457':
        y_train[i]=3

for i in range(len(y_val)):
    # if y_val[i]=='safe':
    #     y_val[i]=0
    if y_val[i]=='CWE121':
        y_val[i]=0
    elif y_val[i]=='CWE191':
        y_val[i]=1
    elif y_val[i]=='CWE401':
        y_val[i]=2
    elif y_val[i]=='CWE457':
        y_val[i]=43

In [47]:
print(set   (y_train))
print(set   (y_val))

{0, 1, 2, 3}
{0, 1, 2, 43}


In [48]:
#convert y_train and y_val to int64
y_train = y_train.astype('int64')
y_val = y_val.astype('int64')

In [49]:
#print shape and type of data
print("X_train shape: ", X_train.shape)
print("y_train shape: ", y_train.shape)
print("X_val shape: ", X_val.shape)
print("y_val shape: ", y_val.shape)


X_train shape:  (13128, 64)
y_train shape:  (13128,)
X_val shape:  (2626, 64)
y_val shape:  (2626,)


In [50]:
X_train_tensor = torch.from_numpy(X_train)
y_train_tensor = torch.from_numpy(y_train)
X_val_tensor = torch.from_numpy(X_val)
y_val_tensor = torch.from_numpy(y_val)
print("X_train_tensor shape: ", X_train_tensor.shape)

X_train_tensor shape:  torch.Size([13128, 64])


In [51]:
print(X_train_tensor.shape)

torch.Size([13128, 64])


In [52]:

# Define the neural network architecture
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(64, 32)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(32, 4)
        self.softmax = nn.Softmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.softmax(x)
        return x

# Define the dataset
class MyDataset(Dataset):
    def __init__(self, data, labels):
        self.data = torch.tensor(data, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx], self.labels[idx]

# Prepare the data
data =X_train_tensor
labels =y_train_tensor
dataset = MyDataset(data, labels)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

# Initialize the model, loss function, and optimizer
model = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001)

# Train the model
for epoch in range(20):
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(dataset)
    epoch_acc = 100 * correct / total
    
    print('Epoch [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'.format(epoch+1, 10, epoch_loss, epoch_acc))

C:\Users\dell\AppData\Local\Temp\ipykernel_10116\3450147574.py:20: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.data = torch.tensor(data, dtype=torch.float32)
C:\Users\dell\AppData\Local\Temp\ipykernel_10116\3450147574.py:21: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.labels = torch.tensor(labels, dtype=torch.long)


Epoch [1/10], Loss: 1.3804, Accuracy: 51.91%
Epoch [2/10], Loss: 1.3751, Accuracy: 52.11%
Epoch [3/10], Loss: 1.3696, Accuracy: 46.10%
Epoch [4/10], Loss: 1.3638, Accuracy: 45.25%
Epoch [5/10], Loss: 1.3576, Accuracy: 45.25%
Epoch [6/10], Loss: 1.3510, Accuracy: 45.25%
Epoch [7/10], Loss: 1.3438, Accuracy: 45.25%
Epoch [8/10], Loss: 1.3359, Accuracy: 45.25%
Epoch [9/10], Loss: 1.3272, Accuracy: 45.25%
Epoch [10/10], Loss: 1.3178, Accuracy: 45.25%
Epoch [11/10], Loss: 1.3074, Accuracy: 45.25%
Epoch [12/10], Loss: 1.2961, Accuracy: 45.25%
Epoch [13/10], Loss: 1.2839, Accuracy: 45.25%
Epoch [14/10], Loss: 1.2709, Accuracy: 45.25%
Epoch [15/10], Loss: 1.2573, Accuracy: 45.25%
Epoch [16/10], Loss: 1.2434, Accuracy: 45.25%
Epoch [17/10], Loss: 1.2296, Accuracy: 45.25%
Epoch [18/10], Loss: 1.2161, Accuracy: 45.25%
Epoch [19/10], Loss: 1.2033, Accuracy: 45.25%
Epoch [20/10], Loss: 1.1914, Accuracy: 45.25%
